# Data Engineering at NimbusMegaMart

**From:** Sarah Kim, CTO @ NimbusMegaMart  
**To:** Data Platform Team  
**Subject:** Daily JSON event rollups

This notebook implements the requested daily KPI rollups for NimbusMegaMart's JSON event data, including:
- Daily aggregations by country × category
- Revenue calculations and 7-day rolling windows
- Partitioned Parquet output
- Repartitioning performance experiments

**Requirements:**
- Process JSON event data using Spark DataFrame API
- Create schemas with StructType/StructField
- Implement broadcast joins to minimize shuffles
- Generate daily KPIs with rolling revenue calculations
- Experiment with repartitioning strategies for performance optimization

## Section 1: Setup and Environment Configuration

In [ ]:
# Initialize Spark session with appropriate configurations
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, MapType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import os

# Create Spark session with optimized configurations
spark = SparkSession.builder \
    .appName("NimbusMegaMart-DataEngineering") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Set log level to reduce noise
spark.sparkContext.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"Number of cores: {spark.sparkContext.defaultParallelism}")
print("Spark session initialized successfully!")

## Section 2: Define Data Schemas

Define StructType schemas for all three datasets to ensure proper data type casting and validation.

In [ ]:
# Define schema for events dataset
events_schema = StructType([
    StructField("event_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("item_id", StringType(), True),
    StructField("event_type", StringType(), True),
    StructField("ts", LongType(), True),  # Unix timestamp
    StructField("props", MapType(StringType(), StringType()), True)  # Flexible properties map
])

# Define schema for users dataset
users_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("registration_date", StringType(), True)
])

# Define schema for items dataset
items_schema = StructType([
    StructField("item_id", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("brand", StringType(), True),
    StructField("name", StringType(), True)
])

print("Schemas defined successfully!")
print(f"Events schema: {len(events_schema.fields)} fields")
print(f"Users schema: {len(users_schema.fields)} fields") 
print(f"Items schema: {len(items_schema.fields)} fields")

## Section 3: Read Datasets with Schema Validation

Load the three datasets from JSON files using the predefined schemas.

In [ ]:
# Read datasets with schemas
events_df = spark.read.schema(events_schema).json("data/events.jsonl")
users_df = spark.read.schema(users_schema).json("data/users.jsonl")
items_df = spark.read.schema(items_schema).json("data/items.jsonl")

print("Datasets loaded successfully!")
print("Sample records from each dataset:")
print("\n--- Events Dataset ---")
events_df.show(3, truncate=False)
print("\n--- Users Dataset ---")
users_df.show(3, truncate=False)
print("\n--- Items Dataset ---")
items_df.show(3, truncate=False)

## Section 4: Data Exploration and Partition Analysis

Display record counts and partition numbers for each DataFrame to understand data distribution.

In [ ]:
# Display record counts and partition information
print("=== DATASET STATISTICS ===")
print(f"Events: {events_df.count():,} records, {events_df.rdd.getNumPartitions()} partitions")
print(f"Users: {users_df.count():,} records, {users_df.rdd.getNumPartitions()} partitions")
print(f"Items: {items_df.count():,} records, {items_df.rdd.getNumPartitions()} partitions")

print("\n=== EVENTS DATASET DETAILS ===")
print("Event type distribution:")
events_df.groupBy("event_type").count().orderBy(desc("count")).show()

print("Events by day:")
events_df.select(from_unixtime(col("ts"), "yyyy-MM-dd").alias("date")) \
    .groupBy("date").count() \
    .orderBy("date").show()

## Section 5: Add Timestamp and Date Columns

Transform the 'ts' column to create 'timestamp' and 'date' columns using Spark functions.

In [ ]:
# Add timestamp and date columns to events DataFrame
events_with_time = events_df.withColumn("timestamp", to_timestamp(from_unixtime(col("ts")))) \
                            .withColumn("date", to_date(from_unixtime(col("ts"))))

print("Added timestamp and date columns to events DataFrame")
print("Sample records with new columns:")
events_with_time.select("event_id", "ts", "timestamp", "date", "event_type").show(5, truncate=False)

print("\nDate range in the dataset:")
events_with_time.select(min("date").alias("start_date"), max("date").alias("end_date")).show()

## Section 6: Calculate Revenue Column

Create a 'revenue' column using when().otherwise() logic based on event type and props.

In [ ]:
# Create revenue column using when().otherwise() logic
events_with_revenue = events_with_time.withColumn(
    "revenue",
    when(col("event_type") == "purchase", col("props.price").cast(DoubleType()))
    .otherwise(lit(0.0))
)

print("Added revenue column to events DataFrame")
print("Sample records with revenue column:")
events_with_revenue.select("event_id", "event_type", "props", "revenue").show(10, truncate=False)

print("\nRevenue statistics:")
events_with_revenue.select(
    sum("revenue").alias("total_revenue"),
    avg("revenue").alias("avg_revenue"),
    count(when(col("revenue") > 0, True)).alias("revenue_events"),
    min("revenue").alias("min_revenue"),
    max("revenue").alias("max_revenue")
).show()

print("Revenue distribution by event type:")
events_with_revenue.groupBy("event_type") \
    .agg(sum("revenue").alias("total_revenue"), 
         count("*").alias("event_count")) \
    .orderBy(desc("total_revenue")).show()

## Section 7: Filter Negative Revenue Records

Remove rows with negative revenue values to ensure data quality.

In [ ]:
# Check for negative revenue records
negative_revenue_count = events_with_revenue.filter(col("revenue") < 0).count()
total_records_before = events_with_revenue.count()

print(f"Records with negative revenue: {negative_revenue_count:,}")
print(f"Total records before filtering: {total_records_before:,}")

# Filter out negative revenue records
events_clean = events_with_revenue.filter(col("revenue") >= 0)
total_records_after = events_clean.count()

print(f"Total records after filtering: {total_records_after:,}")
print(f"Records removed: {total_records_before - total_records_after:,}")

# Verify no negative revenues remain
print("\nRevenue range after filtering:")
events_clean.select(min("revenue").alias("min_revenue"), max("revenue").alias("max_revenue")).show()

## Section 8: Broadcast Joins for Events, Items, and Users

Perform efficient joins using broadcast() function to minimize shuffles.

In [ ]:
# Broadcast joins to minimize shuffles
# 
# BROADCAST JOIN EXPLANATION:
# Broadcast joins are used when one dataset is much smaller than another.
# The smaller dataset is sent to every executor node, eliminating the need
# for shuffle operations. This is highly efficient for our use case because:
# 1. Items dataset (200 records) << Events dataset (50,000 records)  
# 2. Users dataset (1,000 records) << Events dataset (50,000 records)
# 3. By broadcasting the smaller datasets, we avoid expensive shuffles that
#    would normally be required to co-locate matching records across partitions
# 4. This reduces network I/O and improves join performance significantly

print("=== PERFORMING BROADCAST JOINS ===")
print("Broadcasting smaller datasets (items & users) to avoid shuffles...")

# First join: events with items (broadcast items)
events_with_items = events_clean.join(
    broadcast(items_df),
    events_clean.item_id == items_df.item_id,
    "inner"
).drop(items_df.item_id)  # Remove duplicate item_id column

print(f"Events + Items: {events_with_items.count():,} records")

# Second join: events_with_items with users (broadcast users)  
events_enriched = events_with_items.join(
    broadcast(users_df),
    events_with_items.user_id == users_df.user_id,
    "inner"
).drop(users_df.user_id)  # Remove duplicate user_id column

print(f"Final enriched dataset: {events_enriched.count():,} records")

print("\nSample of enriched dataset:")
events_enriched.select("event_id", "event_type", "country", "category", "revenue", "date") \
    .show(5, truncate=False)

## Section 9: Daily KPI Aggregation by Country and Category

Group data by date, country, and category to calculate key performance indicators.

In [ ]:
# Calculate daily KPIs grouped by date, country, and category
daily_kpi = events_enriched.groupBy("date", "country", "category") \
    .agg(
        count("*").alias("events_total"),
        count(when(col("event_type") == "purchase", True)).alias("purchases"),
        sum("revenue").alias("revenue"),
        countDistinct("user_id").alias("unique_users")
    ) \
    .orderBy("date", "country", "category")

print("=== DAILY KPI AGGREGATION RESULTS ===")
print(f"Total daily KPI records: {daily_kpi.count():,}")

print("\nSample daily KPI data:")
daily_kpi.show(10, truncate=False)

print("\nKPI summary statistics:")
daily_kpi.select(
    sum("events_total").alias("total_events"),
    sum("purchases").alias("total_purchases"), 
    sum("revenue").alias("total_revenue"),
    sum("unique_users").alias("total_unique_users_sum"),
    avg("events_total").alias("avg_daily_events"),
    avg("revenue").alias("avg_daily_revenue")
).show()

print("\nTop performing country-category combinations by revenue:")
daily_kpi.groupBy("country", "category") \
    .agg(sum("revenue").alias("total_revenue")) \
    .orderBy(desc("total_revenue")) \
    .show(10)

## Section 10: 7-Day Rolling Revenue Window Function

Implement window functions to calculate 7-day rolling revenue sums.

In [ ]:
# Define window for 7-day rolling revenue calculation
# Partition by country and category, order by date
# Frame: current row + 6 preceding rows (7 days total)
window_7d = Window.partitionBy("country", "category") \
                  .orderBy("date") \
                  .rowsBetween(-6, 0)

# Add 7-day rolling revenue column
daily_kpi_with_rolling = daily_kpi.withColumn(
    "revenue_7d", 
    sum("revenue").over(window_7d)
)

print("=== 7-DAY ROLLING REVENUE CALCULATION ===")
print("Added revenue_7d column with 7-day rolling window")

print("\nSample data with 7-day rolling revenue:")
daily_kpi_with_rolling.select("date", "country", "category", "revenue", "revenue_7d") \
    .orderBy("country", "category", "date") \
    .show(15, truncate=False)

print("\nExample: 7-day rolling revenue for US Electronics:")
daily_kpi_with_rolling.filter(
    (col("country") == "US") & (col("category") == "electronics")
).select("date", "revenue", "revenue_7d") \
.orderBy("date").show()

print("Final schema:")
daily_kpi_with_rolling.printSchema()

## Section 11: Write Partitioned Parquet Files

Save the daily KPI DataFrame as partitioned Parquet files by date.